In [ ]:
## BERT

In [ ]:
!pip install transformers torch

In [ ]:
import pandas as pd

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
import torch

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

In [ ]:
from sklearn.preprocessing import LabelEncoder, label_binarize

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from itertools import cycle

In [ ]:
import numpy as np

In [ ]:
# Encoding labels ('positive', 'negative', 'neutral') to numerical values

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df['label'] = label_encoder.fit_transform(df['class'])

In [ ]:
# Tokenization and Data Preparation (using cleaned text)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
encoded_data = tokenizer.batch_encode_plus(

In [ ]:
    df['cleaned_text'].tolist(),

In [ ]:
    add_special_tokens=True,

In [ ]:
    return_attention_mask=True,

In [ ]:
    padding='max_length',

In [ ]:
    max_length=128,  # Adjust if your sentences are longer

In [ ]:
    truncation=True,